In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.utils import image_dataset_from_directory

In [2]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [3]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
gpu

True

In [4]:
train = image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.


2023-12-09 23:48:40.365401: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-09 23:48:40.365438: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-09 23:48:40.365447: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-09 23:48:40.365490: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-09 23:48:40.365508: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
validation = image_dataset_from_directory(
    directory='test', 
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 5000 files belonging to 2 classes.


In [6]:
#NORMALIZATION
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

In [7]:
train = train.map(process)
validation = validation.map(process)

In [8]:
model = Sequential()

In [9]:
# create CNN model

model.add(Conv2D(20,kernel_size=(3,3),padding='same',activation='tanh',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(120,kernel_size=(3,3),padding='same',activation='tanh'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(180,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(90,activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(30,activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(15,activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1,activation='sigmoid'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 20)      560       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 20)      80        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 20)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 120)     21720     
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 120)     480       
 chNormalization)                                                
                                                        

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
his = model.fit(train, epochs=20, validatio_data=validation)

Epoch 1/20


2023-12-09 23:48:41.903373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 146s 223ms/step - loss: 0.6255 - accuracy: 0.6467 - val_loss: 0.5598 - val_accuracy: 0.7132
Epoch 2/20
625/625 [==============================] - 138s 221ms/step - loss: 0.5654 - accuracy: 0.7018 - val_loss: 0.5202 - val_accuracy: 0.7442
Epoch 3/20
625/625 [==============================] - 139s 222ms/step - loss: 0.5194 - accuracy: 0.7445 - val_loss: 0.5590 - val_accuracy: 0.7114
Epoch 4/20
625/625 [==============================] - 142s 228ms/step - loss: 0.4702 - accuracy: 0.7740 - val_loss: 0.4667 - val_accuracy: 0.7830
Epoch 5/20
625/625 [==============================] - 138s 221ms/step - loss: 0.4175 - accuracy: 0.8120 - val_loss: 0.8921 - val_accuracy: 0.5582
Epoch 6/20
625/625 [==============================] - 141s 225ms/step - loss: 0.3772 - accuracy: 0.8338 - val_loss: 0.4778 - val_accuracy: 0.7782
Epoch 7/20
625/625 [==============================] - 139s 221ms/step - loss: 0.3305 - accuracy: 0.8597 - val_loss: 0.6761 - val_accura

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(his.history['accuracy'],color='red',label='train')
plt.plot(his.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
import cv2

In [ ]:
test = cv2.imread('dog5.png')

In [ ]:
plt.imshow(test)

In [ ]:
# test = cv2.resize(test, (256,256))

In [ ]:
# plt.imshow(test)

In [ ]:
# test = test.reshape((1,256,256,3))

In [ ]:
test = cv2.imread('cat3.png')
test = cv2.resize(test, (256,256))
test = test.reshape((1,256,256,3))

In [ ]:
print(model.predict(test))
if model.predict(test) > 0.6:
    print("Dog")
elif model.predict(test) <= 0.6:
    print("Cat")

In [ ]:
if model.predict(test) > 0.5:
    print("Dog")
elif model.predict(test) <= 0.5:
    print("Cat")